In [4]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [5]:
def visualize_tensor(tensor, title='Tensor Visualization'):
    """
    Визуализирует PyTorch тензор с помощью Matplotlib.

    Параметры:
    - tensor (torch.Tensor): Тензор изображения. Должен быть размерностью (C, H, W) или (H, W) для градаций серого.
    - title (str): Заголовок графика.
    """
    # Проверяем, что тензор находится на GPU и переносим его на CPU
    tensor = tensor.cpu()

    # Если тензор имеет 3 канала (например, RGB), переведём его в numpy массив и изменим порядок осей
    if tensor.dim() == 3 and tensor.size(0) == 3:
        img = tensor.permute(1, 2, 0).numpy()
    elif tensor.dim() == 3:
        # Если тензор имеет больше одного канала (например, выходной слой модели), отображаем только первый канал
        img = tensor[0].numpy()
    elif tensor.dim() == 2:
        # Если тензор имеет только 2D (градации серого)
        img = tensor.numpy()
    else:
        raise ValueError("Тензор должен быть размерности (C, H, W) или (H, W).")

    # Проверяем и нормализуем изображение для корректного отображения
    if img.min() < 0 or img.max() > 1:
        img = (img - img.min()) / (img.max() - img.min())

    # Отображаем изображение
    plt.imshow(img, cmap='gray' if img.ndim == 2 else None)
    plt.title(title)
    plt.axis('off')  # Не показывать оси
    plt.show()


In [ ]:
def crop_and_concat(down_tensor, up_tensor):
    crop_height = (up_tensor.size(2) - down_tensor.size(2)) // 2
    crop_width = (up_tensor.size(3) - down_tensor.size(3)) // 2
    
    up_tensor_cropped = up_tensor[:, :, crop_height:crop_height + down_tensor.size(2), crop_width:crop_width + down_tensor.size(3)]
    
    return torch.cat((up_tensor_cropped, down_tensor), dim=1)

In [ ]:
class UNetConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvLayer, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # encoder (downsampling)
        # Each enc_conv/dec_conv block should look like this:
        # nn.Sequential(
        #     nn.Conv2d(...),
        #     ... (2 or 3 conv layers with relu and batchnorm),
        # )
        self.enc_conv0 = UNetConvLayer(3, 64)
        self.pool0 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.enc_conv1 = UNetConvLayer(64, 128)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.enc_conv2 = UNetConvLayer(128, 256)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.enc_conv3 = UNetConvLayer(256, 512)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        # bottleneck
        self.bottleneck_conv = UNetConvLayer(512, 1024)

        # decoder (upsampling)
        self.upsample0 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec_conv0 = UNetConvLayer(1024, 512)
        self.upsample1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec_conv1 = UNetConvLayer(512, 256)
        self.upsample2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec_conv2 = UNetConvLayer(256, 128)
        self.upsample3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec_conv3 = UNetConvLayer(128, 64)
        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)


    def forward(self, x):
        # Encoder
        e0 = self.enc_conv0(x)
        e1, idx1 = self.pool0(e0)
        e1 = self.enc_conv1(e1)
        e2, idx2 = self.pool1(e1)
        e2 = self.enc_conv2(e2)
        e3, idx3 = self.pool2(e2)
        e3 = self.enc_conv3(e3)
        e4, idx4 = self.pool3(e3)

        # Bottleneck
        b = self.bottleneck_conv(e4)
        # print(b.size())


        # decoder
        d0 = self.upsample0(b)
        d0 = crop_and_concat(d0, e3)  # concatenate along the channel axis
        d0 = self.dec_conv0(d0)

        d1 = self.upsample1(d0)
        d1 = crop_and_concat(d1, e2)  # concatenate along the channel axis
        d1 = self.dec_conv1(d1)

        d2 = self.upsample2(d1)
        d2 = crop_and_concat(d2, e1)  # concatenate along the channel axis
        d2 = self.dec_conv2(d2)

        d3 = self.upsample3(d2)
        d3 = crop_and_concat(d3, e0)  # concatenate along the channel axis
        d3 = self.dec_conv3(d3)

        d3 = self.final_conv(d3)
        return d3